In [ ]:
#@title imports
!pip install openai
!pip install langchain

In [94]:
pip freeze > requirements.txt

In [2]:
#@title imports
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate,  LLMChain
import os

In [52]:
#@title tempalte definition

template="""You are a English song Lyricist. Famous singers come to you with descriptions of the kind of song they want you to write.
They also give you some examples of the song lyrics based on the description.
Output only the lyrics of the song that you should write. do not output anything else.
The avg number of words in lyrics should be around 500-600 and it should have some chorus and verses.

Format of the examples is:
-- Description: <description of the song>
-- Lyrics : <lyrics of the song>

----------------------------------- EXAMPLES START -----------------------------------
-- Description 1: ```{desc_1}```
-- Lyrics 1: ```{lyrics_1}```

-- Description 2: ```{desc_2}```
-- Lyrics 2: ```{lyrics_2}```

-- Description 3: ```{desc_3}```
-- Lyrics 3: ```{lyrics_3}```

-- Description 4: ```{desc_4}```
-- Lyrics 4: ```{lyrics_4}```

-- Description 5: ```{desc_5}```
-- Lyrics 5: ```{lyrics_5}```

----------------------------------- EXAMPLES END -----------------------------------

Note that the avg number of words in lyrics should be around 500-600 and it should have some repeating chorus and verses.

-- Description: ```{desc_user}```

-- Lyrics :

"""

input_variables = ["desc_1", "lyrics_1", "desc_2", "lyrics_2", "desc_3", "lyrics_3", "desc_4", "lyrics_4", "desc_5", "lyrics_5", "desc_user"] # parameters. for tempalte

In [3]:
#@title read secret from file : "openai_api_key.txt"
with open("openai_key.txt", 'r') as f:
    secret = f.read().strip()

os.environ['OPENAI_API_KEY']= secret

In [4]:
#@title initialize_model function
def initialize_model(template, input_variables):

  # llm = ChatOpenAI(model_name="gpt-3.5-turbo")
  llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k")

  prompt = PromptTemplate(template= template, input_variables= input_variables)

  llm_chain = LLMChain(prompt=prompt, llm=llm)

  return llm_chain


In [5]:
#@title QA_Pipeline_TfIdf { form-width: "20%" }

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import sys



class QA_Pipeline_TfIdf:

    def __init__(self):
        pass

    def normalizeData(self,data,isNormalizeAgainstMax=False):
        if sum(data)==0:
            return data
        if isNormalizeAgainstMax:
          return [float(i)/max(data) for i in data]  # to normalize against max
        else:
          return [float(i)/sum(data) for i in data]  # to normalize to make sum = 1


    def create_tfidf_features(self,data, max_features=5000, max_df=0.85, min_df=2):
        """ Creates a tf-idf matrix for the `data` using sklearn. """
        tfidf_vectorizor = TfidfVectorizer(decode_error='replace', strip_accents='unicode', analyzer='word',
                                        stop_words='english', ngram_range=(1, 1), max_features=max_features,
                                        norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                                        max_df=max_df, min_df=min_df)
        X = tfidf_vectorizor.fit_transform(data)
        print('tfidf matrix successfully created.')
        return X, tfidf_vectorizor

    def calculate_similarity( self,query, top_k=20):
        """ Vectorizes the `query` via `vectorizer` and calculates the cosine similarity of
        the `query` and `allDocuments` (all the documents) and returns the `top_k` similar documents."""

        # Vectorize the query to the same length as documents
        query_vec = self.vectorizer.transform(query)
        # Compute the cosine similarity between query_vec and all the documents
        cosine_similarities = cosine_similarity(self.tfidfTransformed_docs,query_vec).flatten()

        # Sort the similar documents from the most similar to less similar and return the indices
        most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]

        # Sort the similar documents from the most similar to less similar and return the scores
        cosine_similarities = np.sort(cosine_similarities)[:-top_k-1:-1]

        # #normalize scores
        cosine_similarities = self.normalizeData(cosine_similarities,isNormalizeAgainstMax=True)
        return most_similar_doc_indices, cosine_similarities

    def RunTfIdf(self,question,top_n=20, getDocuments=False,data=None):
        '''
        given a question find the top_n most similar documents.
        if document text is also needed then pass getDocuments=True and pass the data which was originally passed to createTfIdfTable
        '''
        top_idx,cosine_similarities = self.calculate_similarity( [question],top_k=top_n)

        if getDocuments:
            if data==None:
                sys.exit('data to be returned but no data provided. data is n')
            retData = [data[i] for i in top_idx]
            return top_idx,cosine_similarities,retData
        else:
            return top_idx,cosine_similarities

    def createTfIdfTable(self,data,maxFeatures=10000):
        self.tfidfTransformed_docs,self.vectorizer = self.create_tfidf_features( data  ,max_features=maxFeatures)
        # features = vectorizer.get_feature_names()

    def get_nearest_songs(self, data_df, text, top_n=5):
        '''
        given a string text, fine the nearest songs descriptions from the data
        return a list of descriptions and lyrics
        '''

        desc_data  = data_df['description'].tolist()


        self.createTfIdfTable(desc_data,maxFeatures=10000)

        top_idx,cosine_similarities,retData  = self.RunTfIdf(text, top_n=top_n, getDocuments=True, data=desc_data)

        # get the descriptions of top_idx from df
        desc = []
        lyrics = []
        titles = []
        ids = []

        df_titles = data_df['title'].tolist()
        df_ids = data_df['id'].tolist()
        df_desc = data_df['description'].tolist()
        df_lyrics = data_df['lyrics_clean_with_newline'].tolist()


        for i in top_idx:
            desc.append(df_desc[i])
            lyrics.append(df_lyrics[i])
            titles.append(df_titles[i])
            ids.append(df_ids[i])

        return ids, titles, desc, lyrics



In [71]:
#@title generate_lyrics function
def generate_lyrics(desc_user, data_df, prompt_template, input_variables, specific_artist=None ):
    '''
        Given the user query/descipriton of the lyrics, return the lyrics of the song
    '''


    template = prompt_template
    input_variables = input_variables



    if specific_artist is None:
        sub_df = data_df

    else:
        # repalce "Format of the examples is:" with "the Lyrics should be in the style of <artist_name>." + "\n Format of the examples is:"
        temp = "Format of the examples is:"
        template = template.replace(temp, "the Lyrics should be in the style of " + specific_artist + ".\n" + temp)

        try:
            sub_df = data_df[data_df['artist']==specific_artist]
            # if this artist has less then 5 entries then get the whole df
            if sub_df.shape[0] < 5:
                print("This artist has less than 5 songs. taking the whole data")
                sub_df = data_df
        except:
            print("Artist wasnt found. taking the whole data")
            sub_df = data_df


    llm_chain = initialize_model(template, input_variables)


    tfidf = QA_Pipeline_TfIdf()

    ids, titles, desc, lyrics = tfidf.get_nearest_songs(sub_df, desc_user)

    params = {}
    params["desc_1"] = desc[0]
    params["desc_2"] = desc[1]
    params["desc_3"] = desc[2]
    params["desc_4"] = desc[3]
    params["desc_5"] = desc[4]
    params["lyrics_1"] = lyrics[0]
    params["lyrics_2"] = lyrics[1]
    params["lyrics_3"] = lyrics[2]
    params["lyrics_4"] = lyrics[3]
    params["lyrics_5"] = lyrics[4]

    params["desc_user"] = desc_user


    model_output = llm_chain.run(params)

    return model_output

# Main functions callings

In [69]:
import pandas as pd

# CSV_FILEPATH = "data/data_clean_en_3.csv"
CSV_FILEPATH = "/content/drive/MyDrive/MusicProject/data_clean_en_3.csv"
df = pd.read_csv(CSV_FILEPATH)
df.shape

(2558, 9)

In [82]:
desc_user = "World war 2 was the worst event in history. Write a songs about it. "

In [83]:
model_output_1 = generate_lyrics(desc_user, df, template, input_variables)

tfidf matrix successfully created.


In [84]:
model_output_eminem = generate_lyrics(desc_user, df, template, input_variables, specific_artist="Eminem")

tfidf matrix successfully created.


In [85]:
model_output_taylor = generate_lyrics(desc_user, df, template, input_variables, specific_artist="Taylor Swift")

tfidf matrix successfully created.


In [86]:
model_output_katy = generate_lyrics(desc_user, df, template, input_variables, specific_artist="Katy Perry")

tfidf matrix successfully created.


In [87]:
model_output_rihanna = generate_lyrics(desc_user, df, template, input_variables, specific_artist="Rihanna")

tfidf matrix successfully created.


In [88]:
print(model_output_1)

In the darkest of times, in a world torn apart
A war like no other, etched in every heart
World War II, the greatest tragedy we've known
A chapter in history that must never be disowned

Chorus:
We remember the fallen, the heroes who fought
Their sacrifices and bravery, will never be forgot
In the ashes of war, we rise above the pain
United we stand, for peace we shall regain

Verse 1:
From the depths of despair, a tyrant had emerged
Spreading hatred and fear, his power fully submerged
Nations divided, armies were deployed
Innocent lives lost, the world left destroyed

Pre-Chorus:
But through the darkness, a light did shine
Courageous souls rose, a mission to redefine
They fought for freedom, for justice and for peace
Their battle cry echoed, their strength would never cease

Chorus:
We remember the fallen, the heroes who fought
Their sacrifices and bravery, will never be forgot
In the ashes of war, we rise above the pain
United we stand, for peace we shall regain

Verse 2:
On the beac

In [89]:
print(model_output_eminem)

In the darkest days of history, a war did unfold
A conflict that shook the world, a story yet untold
World War II, the battle of the nations
A fight for freedom, against evil's creations

From the ashes of despair, heroes did arise
Facing the enemy, with fire in their eyes
They fought for justice, for peace and for love
Their courage and sacrifice, a gift from above

Chorus:
March on, soldiers of the past
In our hearts, your memory will last
Through the smoke and the fire, we shall stand tall
For the heroes of World War II, we'll never let them fall

In the midst of darkness, hope did prevail
As nations united, their strength did unveil
From the beaches of Normandy, to the skies above
The Allied forces fought with bravery and love

But the cost was great, the toll of war immense
Lives forever changed, families left in suspense
Millions lost their lives, their stories left untold
In the annals of history, their names engraved in gold

Chorus:
March on, soldiers of the past
In our hearts

In [90]:
print(model_output_taylor)

In the darkest hour, when the world was torn apart 
A battle raged on, breaking souls and breaking hearts 
The echoes of the guns, the cries of those in pain 
In the midst of the chaos, hope began to wane 

But through the smoke and fire, heroes would arise 
They fought for freedom, they fought for their lives 
In the face of tyranny, they stood strong and proud 
Their courage and resilience, a beacon in the crowd 

(Chorus)
We will remember, the heroes of the past 
Their sacrifice and bravery, forever it will last 
In the depths of darkness, they brought a shining light 
Their legacy lives on, forever burning bright 

(Verse 2)
From the beaches of Normandy, to the skies up above 
The battles were fought, with determination and love 
They gave their all, for a world that's free 
Their sacrifice and valor, for all eternity 

In the trenches they fought, with no end in sight 
But they never gave up, they never lost their fight 
They left their families behind, kissed them one last time 


In [91]:
print(model_output_katy)

In the darkest times, we stood strong and brave 
Through the chaos and the pain, we found a way to save 
Our spirits never wavered, our hope never died 
In the face of destruction, we found the strength to rise 

(Chorus)
Oh, World War 2, a chapter so dark 
But in the midst of the chaos, we left our mark 
We fought for freedom, we fought for peace 
In the depths of despair, our courage did not cease 

Verse 1:
In the trenches we stood, side by side 
Brothers and sisters, our hearts unified 
Through the smoke and the fire, we held our ground 
For our loved ones, our country, we would never back down 

(Chorus)
Oh, World War 2, a chapter so dark 
But in the midst of the chaos, we left our mark 
We fought for freedom, we fought for peace 
In the depths of despair, our courage did not cease 

Verse 2:
From the beaches of Normandy to the skies above 
We faced the enemy with a fierce love 
Our sacrifice was great, our losses profound 
But through it all, our spirit was unbound 

(Chorus)
Oh,

In [92]:
print(model_output_rihanna)

(Verse 1)
In the darkest hour, a storm began to brew
A world engulfed in chaos, everything askew
The battles that were fought, the lives that were lost
World War II, the ultimate cost

(Chorus)
World War II, the darkest of days
Millions of lives lost in so many ways
From the ashes we rise, stronger and free
Remembering the past, the lessons we must see

(Verse 2)
From the trenches to the skies, the fight was everywhere
Soldiers marching forward, with courage they would bear
The bombs that rained down, the cities turned to dust
The world united, in the face of mistrust

(Chorus)
World War II, the darkest of days
Millions of lives lost in so many ways
From the ashes we rise, stronger and free
Remembering the past, the lessons we must see

(Bridge)
But through the darkness, a glimmer of hope appeared
The sacrifices made, the victory that was near
We honor the fallen, the heroes of the past
Their courage and resilience, forever shall it last

(Chorus)
World War II, the darkest of days
Mill